# LGBM

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import datetime as dt
import numpy as np

import pandas as pd
import datetime as dt
import numpy as np

class TrainDataProcessor:
    """I am rewriting this training data processor to process a few more variables differently."""

    def __init__(self, train, revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices, for_testing=False,
                add_log_cols=False):
        self.add_log_cols = add_log_cols
        self.test_orig_dfs = self.get_test_orig_dfs([train, revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices])
        
        self.weather_mapping = self.init_weather_mapping()
        
        if not for_testing:
            self.train = self.init_train(train)
            self.revealed_targets = self.init_revealed_targets(revealed_targets)
            self.client = self.init_client(client)
            
            self.historical_weather = self.init_historical_weather(historical_weather)
            self.forecast_weather = self.init_forecast_weather(forecast_weather)
            self.electricity_prices = self.init_electricity(electricity_prices)
            self.gas_prices = self.init_gas_prices(gas_prices)
            
            self.df_all_cols = self.join_data(self.train, self.revealed_targets, self.client, self.historical_weather, self.forecast_weather, self.electricity_prices, self.gas_prices)
            if self.add_log_cols:
                self.df_all_cols = self.create_log_cols(self.df_all_cols)
            self.df = self.remove_cols(self.df_all_cols)
            
        
    def get_test_orig_dfs(self, dfs):
        for i, df in enumerate(dfs):
            if 'datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.datetime)
                col = 'datetime'
            if 'prediction_datetime' in df.columns:
                df['prediction_datetime'] = pd.to_datetime(df.prediction_datetime)
                col = 'prediction_datetime'
            if 'forecast_date' in df.columns:
                df['forecast_date'] = pd.to_datetime(df['forecast_date'])
                col = 'forecast_date'
            if 'forecast_datetime' in df.columns:
                df['forecast_datetime'] = pd.to_datetime(df['forecast_datetime'])
                col = 'forecast_datetime'
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df.date).dt.date
                col = 'date'

            test_date = df[col].iloc[-1]  # Assuming test is a DataFrame
            start_date = test_date - pd.Timedelta(days=14)
            historical_subset = df[df[col] >= start_date]
            dfs[i] = historical_subset.copy()
        return dfs
        
    def init_train(self, df):
        """Prepares the training data for model training."""
        try:
            df['datetime'] = pd.to_datetime(df.datetime)
        except Exception as e:
            df['datetime'] = pd.to_datetime(df.prediction_datetime)
        df['date'] = df.datetime.dt.date
            
        # df = self.get_data_block_id(df, 'datetime')
        return df
    
    def add_electricity_lag_features(self, df):
        ##### mean from entire last week
        df.set_index('datetime', inplace=True)
        # Use rolling to calculate mean price of the last week
        # The window is 7 days, min_periods can be set as per requirement
        # 'closed' determines which side of the interval is closed; it can be 'right' or 'left'
        df['mean_euros_per_mwh_last_week'] = df['euros_per_mwh'].rolling(window='7D', min_periods=1, closed='right').mean()
        # Shift the results to align with the requirement of lagging
        df['mean_euros_per_mwh_last_week'] = df['mean_euros_per_mwh_last_week'].shift()
        
        ##### mean from last week this hour only
        # Extract hour from datetime
        df['hour'] = df.index.hour

        # Group by hour and apply rolling mean for each group
        hourly_groups = df.groupby('hour')
        dff = hourly_groups['euros_per_mwh'].rolling(window='7D', min_periods=1, closed='right').mean()#.shift()#.reset_index(level=0, drop=True)
        dff = dff.reset_index().set_index('datetime').groupby('hour')['euros_per_mwh'].shift()
        dff = dff.rename('mean_euros_per_mwh_same_hour_last_week')
        df = df.join(dff)
        #### yesterday's power price
        df['yesterdays_euros_per_mwh'] = df['euros_per_mwh'].shift(24)
        
        ### 24h average
        # Calculate the 24-hour rolling average
        df['euros_per_mwh_24h_average_price'] = df['euros_per_mwh'].rolling(window=24, min_periods=1).mean()

        # Resetting the index if needed
        df.reset_index(inplace=True)
        df = df.drop(['forecast_date', 'origin_date', 'hour'], axis=1)
        return df

    def init_electricity(self, df):
        ## LAG = 1 Day
        ## Move forecast datetime ahead by 1 day
        ## change name to datetime
        df['datetime'] = pd.to_datetime(df['forecast_date'])
        df['datetime'] = df['datetime'] + dt.timedelta(days=1)
        # df = self.get_data_block_id(df, 'datetime')
        df = self.add_electricity_lag_features(df)
        return df
    
    def add_historical_weather_lag_features(self, df):
        ##### LATEST WEATHER
        def add_latest_weather(df):
            # Assuming df is your original DataFrame
            # Step 1: Convert datetime to a Datetime Object
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)

            # Step 2: Sorting the Data
            df.sort_values(by=['datetime', 'latitude', 'longitude'], inplace=True)

            # Step 3: Creating a Unique Identifier for each location
            df['location_id'] = df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Step 4: Filtering for 10:00 AM Entries
            df.reset_index(inplace=True)
            df_10am = df[df['datetime'].dt.hour == 10]
            df_10am.set_index('datetime', inplace=True)

            # Step 5: Shifting the Features by 1 day
            lagged_features = df_10am.groupby('location_id').shift(periods=1, freq='D')
            
            # grouped = lagged_features.groupby('county')
            # lagged_features = grouped[weather_features].mean()
            
            
            # Renaming columns to indicate lag
            lagged_features = lagged_features.add_suffix('_hw_lagged')
            lagged_features['location_id'] = lagged_features['location_id_hw_lagged']
            lagged_features.reset_index(inplace=True)
            lagged_features['date'] = lagged_features.datetime.dt.date

            df['date'] = df.datetime.dt.date
            return lagged_features
            # Step 6: Merging Lagged Features with Original DataFrame
            df = df.merge(lagged_features, on=['date', 'location_id'], how='left', suffixes=('', '_hw_lagged'))
            return df
        
        ##### mean from last day
        def add_24h_mean_var(df, weather_features):
            # Calculate the start and end times for each row
            df['start_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=2) + pd.Timedelta(hours=11)
            df['end_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=1) + pd.Timedelta(hours=10)
            df['time_code'] = df['start_time'].astype(str) +'_' + df['end_time'].astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)
            # print(df.time_code)

            # Create a helper column for grouping
            # If the time is before 11:00 AM, subtract a day
            df['group'] = df['datetime'].apply(lambda dt: dt if dt.time() >= pd.to_datetime('11:00').time() else dt - pd.Timedelta(days=1))
            df['group'] = df['group'].dt.date  # Keep only the date part for grouping
            df['group'] = (pd.to_datetime(df['group']) + pd.Timedelta(hours=11)).astype(str) + '_' + (pd.to_datetime(df['group']) + pd.Timedelta(days=1, hours=10)).astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Now group by this new column
            grouped = df.groupby('group')
            means = grouped[weather_features].mean()
            variances = grouped[weather_features].var()

            # Merge means and variances into the original DataFrame
            my_df = df.merge(means, left_on='time_code', right_on='group', suffixes=('', '_hw_means'), how='left')
            my_df = my_df.merge(variances, left_on='time_code', right_on='group', how='left', suffixes=('', '_hw_variances'))

            return my_df
        
        ##### mean from last day all estonia
        def add_24h_mean_var_estonia(df, weather_features):
            # Calculate the start and end times for each row
            df['start_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=2) + pd.Timedelta(hours=11)
            df['end_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=1) + pd.Timedelta(hours=10)
            df['time_code'] = df['start_time'].astype(str) +'_' + df['end_time'].astype(str)
            # print(df.time_code)

            # Create a helper column for grouping
            # If the time is before 11:00 AM, subtract a day
            df['group'] = df['datetime'].apply(lambda dt: dt if dt.time() >= pd.to_datetime('11:00').time() else dt - pd.Timedelta(days=1))
            df['group'] = df['group'].dt.date  # Keep only the date part for grouping
            df['group'] = (pd.to_datetime(df['group']) + pd.Timedelta(hours=11)).astype(str) + '_' + (pd.to_datetime(df['group']) + pd.Timedelta(days=1, hours=10)).astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Now group by this new column
            grouped = df.groupby('group')
            means = grouped[weather_features].mean()
            variances = grouped[weather_features].var()

            # Merge means and variances into the original DataFrame
            my_df = df.merge(means, left_on='time_code', right_on='group', suffixes=('', '_hw_means_estonia'), how='left')
            my_df = my_df.merge(variances, left_on='time_code', right_on='group', how='left', suffixes=('', '_hw_variances'))

            return my_df

        df['datetime'] = pd.to_datetime(df['datetime'])
        weather_features = df.columns.drop(['datetime', 'latitude', 'longitude'])

        # Apply the function
        df = add_24h_mean_var(df, weather_features)       
        latest = add_latest_weather(df)
        df = df.merge(latest, on=['date', 'location_id'], how='left', suffixes=('', '_hw_lagged'))
        
        return df

    def init_historical_weather(self, df):
        ## LAG: From 11:00 AM 2 days ago to 10:00 AM 1 day ago
        ## What to do? Give most recent weather forecast? Give average over the last day?
        """
        Processes the historical weather data.
        """
        df['datetime'] = pd.to_datetime(df.datetime)
        
        
        
        df = self.add_historical_weather_lag_features(df)
        
        df = df.merge(self.weather_mapping, how='inner', on=('latitude', 'longitude'))
        
        return df

    def init_forecast_weather(self, df):
        ## LAG: DON't ADJUST
        ##      The forecast is from yesterday, but can forecast today, which is 22 hours ahead
        ## Drop any columns where:
        ##                        hours_ahead < 22 and hours_ahead > 45
        ## Then rename forecast_datetime to datetime and join on datetime
        """
        Processes the forecast weather data.
        """
        df['datetime'] = pd.to_datetime(df['forecast_datetime'])
        # keep only datetimes from our relevant period
        df = df[(df['hours_ahead'] < 46) & (df['hours_ahead'] > 21)]
        df['datetime'] = df['datetime'] + dt.timedelta(days=1)
        df = df.merge(self.weather_mapping, how='inner', on=('latitude', 'longitude'))
        return df
    
    def add_gas_prices_lag_features(self, df):
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        # Sort the DataFrame by date, if it's not already sorted
        df.sort_index(inplace=True)

        # Calculate rolling averages for different time windows
        df['lowest_price_3d_avg'] = df['lowest_price_per_mwh'].rolling(window=3).mean()
        df['highest_price_3d_avg'] = df['highest_price_per_mwh'].rolling(window=3).mean()

        df['lowest_price_7d_avg'] = df['lowest_price_per_mwh'].rolling(window=7).mean()
        df['highest_price_7d_avg'] = df['highest_price_per_mwh'].rolling(window=7).mean()

        df['lowest_price_14d_avg'] = df['lowest_price_per_mwh'].rolling(window=14).mean()
        df['highest_price_14d_avg'] = df['highest_price_per_mwh'].rolling(window=14).mean()

        # Reset the index if you want the 'date' column back
        df.reset_index(inplace=True)
        return df

    def init_gas_prices(self, df):
        ## LAG: 1 DAY
        ## Predictions are made from 2 days ago and predict for yesterday
        ## add one day to forecast_date
        ## Rename forecast_date to date, join on date
        """
        Processes the gas prices data.
        Implement the logic to handle gas prices data processing here.
        """
        df['date'] = pd.to_datetime(df['forecast_date']).dt.date
        df['date'] = df['date'] + dt.timedelta(days=1)
        df = self.add_gas_prices_lag_features(df)
        return df
    
    def add_revealed_target_features(self, df):
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['hour'] = df.datetime.dt.hour
        df['day'] = df.datetime.dt.dayofweek
        df.set_index('datetime', inplace=True)
        
        # let me add some new features here too
        # Adding lag features
        # Step 2: Sorting the Data
        df.sort_values(by=['datetime'], inplace=True)

        # Step 3: Creating a Unique Identifier for each location
        df['id'] = df['county'].astype(str) + '_' + df['is_business'].astype(str) + '_' + df['product_type'].astype(str) + '_' + df['is_consumption'].astype(str)
        lagged_features = []
        lagged_hours = []
        ### Defining lagged target features

        for lag_hours in range(1, 24):
            lagged_feature = df.groupby('id').shift(periods=lag_hours, freq='H')
            lagged_features.append(lagged_feature)
            lagged_hours.append(lag_hours)

        for lag_hours in ([i*24 for i in range(1,8)] + [24*11, 24*12]):
            lagged_feature = df.groupby('id').shift(periods=lag_hours, freq='H')
            lagged_features.append(lagged_feature)
            lagged_hours.append(lag_hours)
            
        df.reset_index(inplace=True)
        for lagged_feature, lag_hours in zip(lagged_features, lagged_hours):
            lagged_feature.reset_index(inplace=True)
            lagged_feature.dropna(inplace=True)
            df = df.merge(lagged_feature[['datetime', 'target', 'id']], on=['id', 'datetime'], how='left', suffixes=('', f'_lag_{lag_hours}h'))

        df.set_index('datetime', inplace=True)
        
        

        window_size = 7
        # Group by the specified columns and then apply the rolling mean
        grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption'])
        df['target_rolling_avg_24h'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption', 'hour'])
        df['target_rolling_avg_hour_7d'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_avg_hour_hour_day_4w'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption'])
        df['target_rolling_allp_avg_24h'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption', 'hour'])
        df['target_rolling_allp_avg_hour_7d'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption', 'hour', 'day'])
        df['target_rolling_allp_avg_hour_hour_day_4w'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())
        
        #All of estonia
        grouped = df.groupby(['is_business', 'product_type', 'is_consumption'])
        df['target_rolling_avg_24h_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'product_type', 'is_consumption', 'hour'])
        df['target_rolling_avg_hour_7d_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['is_business', 'product_type', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_avg_hour_hour_day_4w_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'is_consumption'])
        df['target_rolling_allp_avg_24h_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'is_consumption', 'hour'])
        df['target_rolling_allp_avg_hour_7d_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['is_business', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_allp_avg_hour_hour_day_4w_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())
        
        df = df.drop(['hour', 'day'], axis=1)

        return df
    
    def init_revealed_targets(self, df):
        df['datetime'] = pd.to_datetime(df.datetime)
        df['datetime'] = df['datetime'] + dt.timedelta(days=2)
        df = self.add_revealed_target_features(df)
        return df
    
    def init_client(self, df):
        ## LAG: 2 days
        ## Add 2 days to date, join on date
        df['date'] = pd.to_datetime(df.date).dt.date
        df['date'] = df['date'] + dt.timedelta(days=2)
        # df = self.get_data_block_id(df, 'date')
        return df

    def init_weather_mapping(self):
        # https://www.kaggle.com/code/tsunotsuno/enefit-eda-baseline/notebook#Baseline
        county_point_map = {
            0: (59.4, 24.7), # "HARJUMAA"
            1 : (58.8, 22.7), # "HIIUMAA"
            2 : (59.1, 27.2), # "IDA-VIRUMAA"
            3 : (58.8, 25.7), # "JÄRVAMAA"
            4 : (58.8, 26.2), # "JÕGEVAMAA"
            5 : (59.1, 23.7), # "LÄÄNE-VIRUMAA"
            6 : (59.1, 23.7), # "LÄÄNEMAA"
            7 : (58.5, 24.7), # "PÄRNUMAA"
            8 : (58.2, 27.2), # "PÕLVAMAA"
            9 : (58.8, 24.7), # "RAPLAMAA"
            10 : (58.5, 22.7),# "SAAREMAA"
            11 : (58.5, 26.7),# "TARTUMAA"
            12 : (58.5, 25.2),# "UNKNOWNN" (center of the map)
            13 : (57.9, 26.2),# "VALGAMAA"
            14 : (58.2, 25.7),# "VILJANDIMAA"
            15 : (57.9, 27.2) # "VÕRUMAA"
        }
        # Convert the dictionary to a list of tuples
        data = [(county_code, lat, lon) for county_code, (lat, lon) in county_point_map.items()]

        # Create DataFrame
        df = pd.DataFrame(data, columns=['county', 'latitude', 'longitude'])
        
        return df
    
    def add_date_features(self, df):
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        df['hour'] = df['datetime'].dt.hour
        df['quarter'] = df['datetime'].dt.quarter
        df['day_of_week'] = df['datetime'].dt.day_of_week
        df['day_of_year'] = df['datetime'].dt.dayofyear
        df['week_of_year'] = df['datetime'].dt.isocalendar().week
        df['is_weekend'] = df['datetime'].dt.day_of_week >= 5
        df['is_month_start'] = df['datetime'].dt.is_month_start
        df['is_month_end'] = df['datetime'].dt.is_month_end
        df['is_quarter_start'] = df['datetime'].dt.is_quarter_start
        df['is_quarter_end'] = df['datetime'].dt.is_quarter_end
        df['is_year_start'] = df['datetime'].dt.is_year_start
        df['is_year_end'] = df['datetime'].dt.is_year_end
        df['season'] = df['datetime'].dt.month % 12 // 3 + 1
        df['hour_sin'] = np.sin(df['datetime'].dt.hour * (2. * np.pi / 24))
        df['hour_cos'] = np.cos(df['datetime'].dt.hour * (2. * np.pi / 24))
        # Calculate sin and cos for day of year
        days_in_year = 365.25  # accounts for leap year
        df['day_of_year_sin'] = np.sin((df['day_of_year'] - 1) * (2 * np.pi / days_in_year))
        df['day_of_year_cos'] = np.cos((df['day_of_year'] - 1) * (2 * np.pi / days_in_year))
        return df
    
    def add_ee_holidays(self, df):
        import holidays
        # Define Estonia public holidays
        ee_holidays = holidays.CountryHoliday('EE')
        
        print(df['date'].isna().sum())
        
        def find_problem(x):
            try:
                return x in ee_holidays
            except Exception as e:
                print(x)
                raise e

        # Function to check if the date is a holiday
        df['is_ee_holiday'] = df['date'].apply(lambda x: x in ee_holidays)

        return df
    
    def create_log_cols(self, df):
        log_cols = ['target_lag_1h', 'target_lag_2h', 'target_lag_3h', 'target_lag_4h',
       'target_lag_5h', 'target_lag_6h', 'target_lag_7h', 'target_lag_8h',
       'target_lag_9h', 'target_lag_10h', 'target_lag_11h', 'target_lag_12h',
       'target_lag_13h', 'target_lag_14h', 'target_lag_15h', 'target_lag_16h',
       'target_lag_17h', 'target_lag_18h', 'target_lag_19h', 'target_lag_20h',
       'target_lag_21h', 'target_lag_22h', 'target_lag_23h', 'target_lag_24h',
       'target_lag_48h', 'target_lag_72h', 'target_lag_96h', 'target_lag_120h',
       'target_lag_144h', 'target_lag_168h', 'target_lag_264h',
       'target_lag_288h', 'eic_count', 'installed_capacity', 'temperature', 'dewpoint', 'rain',
       'snowfall', 'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'temperature_hw_means', 'dewpoint_hw_means',
       'rain_hw_means', 'snowfall_hw_means', 'surface_pressure_hw_means',
       'cloudcover_total_hw_means', 'cloudcover_low_hw_means',
       'cloudcover_mid_hw_means', 'cloudcover_high_hw_means',
       'windspeed_10m_hw_means', 'winddirection_10m_hw_means',
       'shortwave_radiation_hw_means', 'direct_solar_radiation_hw_means',
       'diffuse_radiation_hw_means', 'temperature_hw_variances',
       'dewpoint_hw_variances', 'rain_hw_variances', 'snowfall_hw_variances',
       'surface_pressure_hw_variances', 'cloudcover_total_hw_variances',
       'cloudcover_low_hw_variances', 'cloudcover_mid_hw_variances',
       'cloudcover_high_hw_variances', 'windspeed_10m_hw_variances',
       'winddirection_10m_hw_variances', 'shortwave_radiation_hw_variances',
       'direct_solar_radiation_hw_variances', 'diffuse_radiation_hw_variances',
       'temperature_hw_lagged', 'dewpoint_hw_lagged', 'rain_hw_lagged',
       'snowfall_hw_lagged', 'surface_pressure_hw_lagged',
       'cloudcover_total_hw_lagged', 'cloudcover_low_hw_lagged', 'cloudcover_mid_hw_lagged',
       'cloudcover_high_hw_lagged', 'windspeed_10m_hw_lagged',
       'winddirection_10m_hw_lagged', 'shortwave_radiation_hw_lagged',
       'direct_solar_radiation_hw_lagged', 'diffuse_radiation_hw_lagged',
       'temperature_hw_means_hw_lagged', 'dewpoint_hw_means_hw_lagged',
       'rain_hw_means_hw_lagged', 'snowfall_hw_means_hw_lagged',
       'surface_pressure_hw_means_hw_lagged',
       'cloudcover_total_hw_means_hw_lagged',
       'cloudcover_low_hw_means_hw_lagged',
       'cloudcover_mid_hw_means_hw_lagged',
       'cloudcover_high_hw_means_hw_lagged',
       'windspeed_10m_hw_means_hw_lagged',
       'winddirection_10m_hw_means_hw_lagged',
       'shortwave_radiation_hw_means_hw_lagged',
       'direct_solar_radiation_hw_means_hw_lagged',
       'diffuse_radiation_hw_means_hw_lagged',
       'temperature_hw_variances_hw_lagged', 'dewpoint_hw_variances_hw_lagged',
       'rain_hw_variances_hw_lagged', 'snowfall_hw_variances_hw_lagged',
       'surface_pressure_hw_variances_hw_lagged',
       'cloudcover_total_hw_variances_hw_lagged',
       'cloudcover_low_hw_variances_hw_lagged',
       'cloudcover_mid_hw_variances_hw_lagged',
       'cloudcover_high_hw_variances_hw_lagged',
       'windspeed_10m_hw_variances_hw_lagged',
       'winddirection_10m_hw_variances_hw_lagged',
       'shortwave_radiation_hw_variances_hw_lagged',
       'direct_solar_radiation_hw_variances_hw_lagged',
       'diffuse_radiation_hw_variances_hw_lagged', 'temperature_fw', 'dewpoint_fw', 'cloudcover_high_fw',
       'cloudcover_low_fw', 'cloudcover_mid_fw', 'cloudcover_total_fw',
       '10_metre_u_wind_component', '10_metre_v_wind_component',
       'direct_solar_radiation_fw', 'surface_solar_radiation_downwards',
       'snowfall_fw', 'total_precipitation', 'euros_per_mwh', 'mean_euros_per_mwh_last_week',
       'mean_euros_per_mwh_same_hour_last_week', 'yesterdays_euros_per_mwh',
       'euros_per_mwh_24h_average_price', 'lowest_price_per_mwh',
       'highest_price_per_mwh', 'lowest_price_3d_avg', 'highest_price_3d_avg',
       'lowest_price_7d_avg', 'highest_price_7d_avg', 'lowest_price_14d_avg',
       'highest_price_14d_avg']
        
        log_cols = [col for col in log_cols if col in df.columns]
        
        dff = np.log1p(df[log_cols] )
        dff.rename(columns={col: col + "_log" for col in log_cols}, inplace=True)
        return pd.concat([df, dff], axis=1)
        
    
    def remove_cols(self, df):
        col_list = ['datetime',
                   'row_id',
                   'prediction_unit_id',
                    'date_train',
                    'hour_part',
                   'date_client',
                    'forecast_date_elec_price',
                    'origin_date_elec_price',
                    'forecast_date_gas_price',
                    'origin_date_gas_price',
                    'datetime_hist_weath',
                   'hour_part_hist_weath_latest',
                    'datetime_hist_weath_latest',
                   'origin_datetime',
                   'hour_part_fore_weath',
                    'datetime',
                    'id',
                     'data_block_id',
                     'row_id',
                     'prediction_unit_id',
                     'date',
                    'data_block_id_rt',
                     'row_id_rt',
                     'prediction_unit_id_rt',
                    'data_block_id_client',
                    'latitude',
                     'longitude',
                     'data_block_id_hw',
                    'start_time',
                     'end_time',
                     'time_code',
                     'group',
                    'data_block_id_hw_means',
                    'data_block_id_hw_variances',
                     'location_id',
                     'date_hw',
                     'datetime_hw_lagged',
                    'latitude_hw_lagged',
                     'longitude_hw_lagged',
                     'data_block_id_hw_lagged',
                     'start_time_hw_lagged',
                     'end_time_hw_lagged',
                     'time_code_hw_lagged',
                     'group_hw_lagged',
                    'data_block_id_hw_means_hw_lagged',
                    'data_block_id_hw_variances_hw_lagged',
                    'location_id_hw_lagged',
                     'latitude_fw',
                     'longitude_fw',
                     'origin_datetime',
                    'data_block_id_fw',
                     'forecast_datetime',
                    'data_block_id_elec',
                    'forecast_date',
                    'origin_date',
                     'data_block_id_gasp',
                   ]
        columns_to_drop = [col for col in col_list if col in df.columns]
        df = df.drop(columns_to_drop, axis=1)
        return df
    
    def remove_test_cols(self, df):
        col_list = ['datetime',
                   'prediction_unit_id',
                    'date_train',
                    'hour_part',
                   'date_client',
                    'forecast_date_elec_price',
                    'origin_date_elec_price',
                    'forecast_date_gas_price',
                    'origin_date_gas_price',
                    'datetime_hist_weath',
                   'hour_part_hist_weath_latest',
                    'datetime_hist_weath_latest',
                   'origin_datetime',
                   'hour_part_fore_weath',
                    'datetime',
                     'data_block_id',
                     'row_id',
                     'prediction_unit_id',
                     'date',
                    'data_block_id_rt',
                     'row_id_rt',
                     'prediction_unit_id_rt',
                    'data_block_id_client',
                    'latitude',
                     'longitude',
                     'data_block_id_hw',
                    'start_time',
                     'end_time',
                     'time_code',
                     'group',
                    'data_block_id_hw_means',
                    'data_block_id_hw_variances',
                     'location_id',
                     'date_hw',
                     'datetime_hw_lagged',
                    'latitude_hw_lagged',
                     'longitude_hw_lagged',
                     'data_block_id_hw_lagged',
                     'start_time_hw_lagged',
                     'end_time_hw_lagged',
                     'time_code_hw_lagged',
                     'group_hw_lagged',
                    'data_block_id_hw_means_hw_lagged',
                    'data_block_id_hw_variances_hw_lagged',
                    'location_id_hw_lagged',
                     'latitude_fw',
                     'longitude_fw',
                     'origin_datetime',
                    'data_block_id_fw',
                     'forecast_datetime',
                    'data_block_id_elec',
                    'forecast_date',
                    'origin_date',
                     'data_block_id_gasp',
                    'id'
                   ]
        columns_to_drop = [col for col in col_list if col in df.columns]
        df = df.drop(columns_to_drop, axis=1)
        return df
    
    def join_data(self, train, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices):
        df = train
        df = df.merge(revealed_targets, how='left', on=('datetime', 'county', 'is_business', 'product_type', 'is_consumption'), suffixes=('', '_rt'))
        df = df.merge(client, how='left', on=('date', 'county', 'is_business', 'product_type'), suffixes=('', '_client'))
        df = df.merge(historical_weather, how='left', on=('datetime', 'county'), suffixes=('', '_hw'))
        df = df.merge(forecast_weather, how='left', on=('datetime', 'county'), suffixes=('', '_fw'))
        df = df.merge(electricity_prices, how='left', on='datetime', suffixes=('', '_elec'))
        df['date'] = pd.to_datetime(df['date'])
        df = df.merge(gas_prices, how='left', on='date', suffixes=('', '_gasp'))
        df = self.add_date_features(df)
        df = self.add_ee_holidays(df)
        return df
    
    def add_test_data(self, test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices):
        dfs = [test.copy(), revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices]
        for i, df in enumerate(dfs):
            if 'datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.datetime)
                col = 'datetime'
            if 'prediction_datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.prediction_datetime)
                col = 'datetime'
            if 'forecast_date' in df.columns:
                df['forecast_date'] = pd.to_datetime(df['forecast_date'])
                col = 'forecast_date'
            if 'forecast_datetime' in df.columns:
                df['forecast_datetime'] = pd.to_datetime(df['forecast_datetime'])
                col = 'forecast_datetime'
                
            self.test_orig_dfs[i] = pd.concat([ self.test_orig_dfs[i], df ])          
        
        
    
    def process_test_data_timestep(self, test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices):
        #append test data to test data cache
        self.add_test_data(test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices)
        # process test data
        test = self.init_train(self.test_orig_dfs[0])
        revealed_targets = self.init_revealed_targets(self.test_orig_dfs[1])
        client = self.init_client(self.test_orig_dfs[2])
        historical_weather = self.init_historical_weather(self.test_orig_dfs[3])
        forecast_weather = self.init_forecast_weather(self.test_orig_dfs[4])
        electricity_prices = self.init_electricity(self.test_orig_dfs[5])
        gas_prices = self.init_gas_prices(self.test_orig_dfs[6])
        df_all_cols = self.join_data(test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices)
        if self.add_log_cols:
            df_all_cols = self.create_log_cols(df_all_cols)
        df = self.remove_test_cols(df_all_cols)
        return df
        

In [3]:
with open('data_processor_lgbm1.pkl', 'rb') as f:
    data_processor = pickle.load(f)
data_processor.df

,county,is_business,product_type,target,is_consumption,target_rt,target_lag_1h,target_lag_2h,target_lag_3h,target_lag_4h,...,yesterdays_euros_per_mwh_log,euros_per_mwh_24h_average_price_log,lowest_price_per_mwh_log,highest_price_per_mwh_log,lowest_price_3d_avg_log,highest_price_3d_avg_log,lowest_price_7d_avg_log,highest_price_7d_avg_log,lowest_price_14d_avg_log,highest_price_14d_avg_log
0,0,0,1,0.713,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018609,15,1,0,197.233,1,184.072,171.092,168.933,174.920,170.068,...,4.360164,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137
2018610,15,1,1,0.000,0,0.000,0.000,2.501,25.884,83.535,...,4.360164,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137
2018611,15,1,1,28.404,1,38.646,47.690,34.806,29.202,21.654,...,4.360164,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137
2018612,15,1,3,0.000,0,0.000,0.000,4.512,34.657,122.195,...,4.360164,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137


# Testing

For my experimental CV, I want to take the approach of doing a stratified CV by time - splitting the year into 4 different parts, basically testing the model on each season, 3 months at a time. There was something in the kaggle forums that recommended something like this:

Key: 
= -> training data
+ -> CV data

4 splits in time:
1. =============+++
2. ================+++
3. ===================+++
4. ======================+++



The data starts on 2021-09-01 and ends on 2023-05-31

BUT we don't have enough data to do that properly. So, my CV will instead be:


(Thanks chatgpt)

Splitting the period from 2022-09-01 to 2023-05-31 into five equal parts, here are the date ranges for each segment:

#### First Segment:

From 2022-09-01 to 2022-10-24

#### Second Segment:

From 2022-10-25 to 2022-12-17

#### Third Segment:

From 2022-12-18 to 2023-02-09

#### Fourth Segment:

From 2023-02-10 to 2023-04-04

#### Fifth Segment:

From 2023-04-05 to 2023-05-29


In [11]:
def fill_drop_na(df):
    df = df[~df.target.isna()]
    df = df[~df.target_rolling_avg_24h.isna()]
    means = df.mean()
    # For each column, add an indicator column for NA values
    # for col in df.columns:
    #     if df[col].isna().any():
    #         df[f'{col}_is_na'] = df[col].isna()
    df = df.fillna(means)
    return df, means

In [12]:
%%time
processed_df_no_na, means = fill_drop_na(data_processor.df)
processed_df_no_na.isna().sum()

CPU times: total: 1min 18s
Wall time: 3min 7s


county                       0
is_business                  0
product_type                 0
target                       0
is_consumption               0
                            ..
highest_price_3d_avg_log     0
lowest_price_7d_avg_log      0
highest_price_7d_avg_log     0
lowest_price_14d_avg_log     0
highest_price_14d_avg_log    0
Length: 323, dtype: int64

In [13]:
processed_df_no_na['target_installed_capacity'] = processed_df_no_na['target'] / processed_df_no_na['installed_capacity'] * 1000
processed_df_no_na

C:\Users\mskel\AppData\Local\Temp\ipykernel_15120\2244001451.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  processed_df_no_na['target_installed_capacity'] = processed_df_no_na['target'] / processed_df_no_na['installed_capacity'] * 1000


,county,is_business,product_type,target,is_consumption,target_rt,target_lag_1h,target_lag_2h,target_lag_3h,target_lag_4h,...,euros_per_mwh_24h_average_price_log,lowest_price_per_mwh_log,highest_price_per_mwh_log,lowest_price_3d_avg_log,highest_price_3d_avg_log,lowest_price_7d_avg_log,highest_price_7d_avg_log,lowest_price_14d_avg_log,highest_price_14d_avg_log,target_installed_capacity
11712,0,0,1,0.930,0,0.713,274.689353,274.69907,274.708302,274.717501,...,4.695742,3.856510,3.867026,3.848515,3.860660,4.468619,4.590240,4.478612,4.600884,0.975978
11713,0,0,1,123.214,1,96.590,274.689353,274.69907,274.708302,274.717501,...,4.695742,3.856510,3.867026,3.848515,3.860660,4.468619,4.590240,4.478612,4.600884,129.305586
11714,0,0,2,0.000,0,0.000,274.689353,274.69907,274.708302,274.717501,...,4.695742,3.856510,3.867026,3.848515,3.860660,4.468619,4.590240,4.478612,4.600884,0.000000
11715,0,0,2,21.940,1,17.314,274.689353,274.69907,274.708302,274.717501,...,4.695742,3.856510,3.867026,3.848515,3.860660,4.468619,4.590240,4.478612,4.600884,131.850962
11716,0,0,3,1.611,0,2.904,274.689353,274.69907,274.708302,274.717501,...,4.695742,3.856510,3.867026,3.848515,3.860660,4.468619,4.590240,4.478612,4.600884,0.223505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018609,15,1,0,197.233,1,184.072,171.092000,168.93300,174.920000,170.068000,...,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137,318.117742
2018610,15,1,1,0.000,0,0.000,0.000000,2.50100,25.884000,83.535000,...,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137,0.000000
2018611,15,1,1,28.404,1,38.646,47.690000,34.80600,29.202000,21.654000,...,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137,45.482786
2018612,15,1,3,0.000,0,0.000,0.000000,4.51200,34.657000,122.195000,...,4.035147,3.401197,3.555348,3.381675,3.584259,3.416414,3.584471,3.505922,3.631137,0.000000


In [14]:
from datetime import datetime

cv_ranges_corrected = [
    ('2022-09-01', '2022-10-24'), 
    ('2022-10-25', '2022-12-17'), 
    ('2022-12-18', '2023-02-09'), 
    ('2023-02-10', '2023-04-04'), 
    ('2023-04-05', '2023-05-31')
]

# Function to convert a date string into a datetime object
def to_datetime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d')

# Converting the date strings in cv_ranges to datetime objects
datetime_cv_ranges = [(to_datetime(start), to_datetime(end)) for start, end in cv_ranges_corrected]
datetime_cv_ranges

date_filter = data_processor.df_all_cols.date[processed_df_no_na.index]
date_filter

cv1_train = processed_df_no_na[date_filter <= datetime_cv_ranges[0][0]]
cv1_test = processed_df_no_na[(date_filter <= datetime_cv_ranges[0][1]) & (date_filter > datetime_cv_ranges[0][0])]

In [15]:
cv1_train[['year' ,'month', 'day']]

,year,month,day
11712,2021,9,5
11713,2021,9,5
11714,2021,9,5
11715,2021,9,5
11716,2021,9,5
...,...,...,...
1144249,2022,9,1
1144250,2022,9,1
1144251,2022,9,1
1144252,2022,9,1


In [16]:
cv1_test[['year' ,'month', 'day']]

,year,month,day
1144254,2022,9,2
1144255,2022,9,2
1144256,2022,9,2
1144257,2022,9,2
1144258,2022,9,2
...,...,...,...
1315849,2022,10,24
1315850,2022,10,24
1315851,2022,10,24
1315852,2022,10,24


In [17]:
processed_df_no_na[['year', 'month', 'day']]

,year,month,day
11712,2021,9,5
11713,2021,9,5
11714,2021,9,5
11715,2021,9,5
11716,2021,9,5
...,...,...,...
2018609,2023,5,31
2018610,2023,5,31
2018611,2023,5,31
2018612,2023,5,31


## Train 27

GRID SEARCH

In [19]:
%%time

# https://www.kaggle.com/code/chaozhuang/enefit-eda-w-fft-ssa-arima-lgbm?scriptVersionId=156414824#Predictive-Modelling
from lightgbm import LGBMRegressor
import random
import lightgbm as lgb
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

def tune_lgbm_model(base_params, X_train, y_train, n_iter=8, cv=3):
    """
    Tune a LightGBM model based on a base set of parameters.

    :param base_params: Dictionary of base parameters for the model
    :param X_train: Training features
    :param y_train: Training target variable
    :param n_iter: Number of iterations for RandomizedSearchCV
    :param cv: Number of cross-validation folds
    :return: Best estimator and best parameters
    """
    # Parameter distributions for random search
    cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
    cat_features = [c for c in cat_features if c in df_train_data.columns]
    
    param_dist = {
        'learning_rate': sp_uniform(0.005, 0.5),
        'lambda_l1': sp_uniform(0, 5), 
        'lambda_l2': sp_uniform(0, 5), 
        'max_bin': sp_randint(100, 1000),
        'min_data_in_leaf': sp_randint(15, 300),
        'n_estimators': sp_randint(1000, 8500),
        
        'colsample_bytree' : sp_uniform(0.1, 1),
        'colsample_bynode' : sp_uniform(0.1, 1)
        # 'data_sample_strategy' : ['bagging', 'goss'],
        # 'num_leaves': sp_randint(1, 200),
        # 'drop_rate': sp_uniform(0, 1),
        # 'skip_drop': sp_uniform(0, 1),
        # 'min_data_per_group': sp_randint(10, 200),
        # 'max_cat_threshold': sp_randint(10, 100),
        # 'cat_l2': sp_randint(10, 100),
        # 'cat_smooth': sp_randint(10, 100),
    }

    # Create a LightGBM regressor object
    lgb_reg = lgb.LGBMRegressor(**base_params)

    # Create a RandomizedSearchCV object
    random_search = HalvingRandomSearchCV(estimator=lgb_reg, param_distributions=param_dist,
                                       scoring='neg_mean_absolute_error',
                                       cv=cv, random_state=202, verbose=1,
                                         aggressive_elimination= True,
                                         max_resources=20000, min_resources=5)

    # Fit the random search to the data
    random_search.fit(X_train, y_train, categorical_feature=cat_features)

    # Return the best estimator and best parameters
    return random_search.best_estimator_, random_search.best_params_

base_params_p1 = {
    'verbose': -1,
    'metric': 'mae',
    'n_jobs': 32,
    'boosting': 'dart',
    'objective': 'tweedie'
}

i=4
train = processed_df_no_na[(date_filter <= to_datetime('2023-03-30'))]
val = processed_df_no_na[(date_filter > to_datetime('2023-03-30'))]
print(f"Fold {i}")
print(f"Train rows: {len(train)}")
print(f"Val rows: {len(val)}")

target_cols = ['target', 'target_installed_capacity']
drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
            'snowfall_fw', 'snowfall_hw_means']

df_train_target = train[target_cols]
df_train_data = train.drop(drop_cols, axis=1)

df_val_target2 = val[target_cols]
df_val_data2 = val.drop(drop_cols, axis=1)

cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
cat_features = [c for c in cat_features if c in df_train_data.columns]

# Fit the model
best_model, best_params = tune_lgbm_model(base_params_p1, df_train_data, df_train_target["target"])

print("Best parameters of p1:", best_params)

Fold 4
Train rows: 1805830
Val rows: 195264
n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 5
max_resources_: 20000
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 4000
n_resources: 5
Fitting 3 folds for each of 4000 candidates, totalling 12000 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2382 fits failed out of a total of 12000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1173 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\sklearn.py", line 1049, in fit
    super().fit(
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\sklearn.py", line 842, in fit
    self._Booster = train(
  File "C:\Users\mskel\.conda\

----------
iter: 1
n_candidates: 1334
n_resources: 15
Fitting 3 folds for each of 1334 candidates, totalling 4002 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [ -27.99099922  -27.99099922  -27.99099922 ... -189.37142354 -189.37142354
 -189.37142354]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [ -42.96562938  -42.96562938  -42.96562938 ... -246.38080249 -246.38080249
 -246.38080249]
  warnings.warn(


----------
iter: 2
n_candidates: 445
n_resources: 45
Fitting 3 folds for each of 445 candidates, totalling 1335 fits


...

----------
iter: 6
n_candidates: 6
n_resources: 3645
Fitting 3 folds for each of 6 candidates, totalling 18 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [ -27.99099922  -27.99099922  -27.99099922 ... -225.26692846 -106.13336832
  -58.42203297]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [ -42.96562938  -42.96562938  -42.96562938 ... -241.78205781 -104.47065886
   -1.09880887]
  warnings.warn(


----------
iter: 7
n_candidates: 2
n_resources: 10935
Fitting 3 folds for each of 2 candidates, totalling 6 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [-27.99099922 -27.99099922 -27.99099922 ... -58.42203297 -59.65437186
 -57.01501209]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [-42.96562938 -42.96562938 -42.96562938 ...  -1.09880887  -1.28044239
  -1.21133632]
  warnings.warn(


Best parameters of p1: {'colsample_bynode': 0.41017411019547834, 'colsample_bytree': 0.7711664691469922, 'lambda_l1': 0.297793613166748, 'lambda_l2': 0.3614843058449302, 'learning_rate': 0.26360243974444403, 'max_bin': 916, 'min_data_in_leaf': 75, 'n_estimators': 8026}
CPU times: total: 1d 6h 28min 13s
Wall time: 1h 18s


Best Parameter runs:

Best parameters of p1: {'colsample_bynode': 0.41017411019547834, 'colsample_bytree': 0.7711664691469922, 'lambda_l1': 0.297793613166748, 'lambda_l2': 0.3614843058449302, 'learning_rate': 0.26360243974444403, 'max_bin': 916, 'min_data_in_leaf': 75, 'n_estimators': 8026}
CPU times: total: 1d 6h 28min 13s
Wall time: 1h 18s

Best parameters of p1: {'colsample_bynode': 0.3104645962074113, 'colsample_bytree': 0.8812805682576964, 'lambda_l1': 1.4774526938254677, 'lambda_l2': 0.9829140928475261, 'learning_rate': 0.020111960332727104, 'max_bin': 497, 'min_data_in_leaf': 41, 'n_estimators': 2279}

In [20]:
from lightgbm import LGBMRegressor

def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        target_cols = ['target', 'target_installed_capacity']
        drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                    'snowfall_fw', 'snowfall_hw_means']
        
        df_train_target = train[target_cols]
        df_train_data = train.drop(drop_cols, axis=1)
        
        df_val_target2 = val[target_cols]
        df_val_data2 = val.drop(drop_cols, axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        cat_features = [c for c in cat_features if c in df_train_data.columns]
        
        # We leave max_depth as -1
        # Tune num_leaves, default is 31, let's double it
        params = {'colsample_bynode': 0.41017411019547834, 'colsample_bytree': 0.7711664691469922, 'lambda_l1': 0.297793613166748, 'lambda_l2': 0.3614843058449302, 'learning_rate': 0.26360243974444403, 'max_bin': 916, 'min_data_in_leaf': 75, 'n_estimators': 8026, 
                    'metric': 'mae', 'n_jobs': 22, 'boosting': 'dart', 'objective': 'tweedie'}
        clf2 = LGBMRegressor(**params, random_state=42, verbose=0, importance_type='gain')
        clf2.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

        from sklearn.metrics import mean_absolute_error
        
        print("###############   Target   #################")
        y_pred = clf2.predict(df_train_data)
        y_pred
        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f"For fold {i}: Train Mean Absolute Error:", mae)

        y_pred_val = clf2.predict(df_val_data2)
        y_pred_val

        mae = mean_absolute_error(df_val_target2.target, y_pred_val)
        print(f"For fold {i}: Fold Val Mean Absolute Error:", mae)
        
        importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        # display(importance.head(30))
        # display(importance.tail(30))
        print()
        print()

In [21]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.297793613166748, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.297793613166748
[LightGBM] [Warning] lambda_l2 is set=0.3614843058449302, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3614843058449302
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.297793613166748, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.297793613166748
[LightGBM] [Warning] lambda_l2 is set=0.3614843058449302, reg_lambda=0.0 will be ignored. Current va

In [33]:
%%time

# https://www.kaggle.com/code/chaozhuang/enefit-eda-w-fft-ssa-arima-lgbm?scriptVersionId=156414824#Predictive-Modelling
from lightgbm import LGBMRegressor
import random
import lightgbm as lgb
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

def tune_lgbm_model(base_params, X_train, y_train, n_iter=8, cv=3):
    """
    Tune a LightGBM model based on a base set of parameters.

    :param base_params: Dictionary of base parameters for the model
    :param X_train: Training features
    :param y_train: Training target variable
    :param n_iter: Number of iterations for RandomizedSearchCV
    :param cv: Number of cross-validation folds
    :return: Best estimator and best parameters
    """
    # Parameter distributions for random search
    cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
    cat_features = [c for c in cat_features if c in df_train_data.columns]
    
    param_dist = {
        'learning_rate': sp_uniform(0.005, 0.5),
        'lambda_l1': sp_uniform(0, 5), 
        'lambda_l2': sp_uniform(0, 5), 
        'max_bin': sp_randint(100, 1000),
        'min_data_in_leaf': sp_randint(15, 300),
        'n_estimators': sp_randint(1500, 3500),
        
        # 'colsample_bytree' : sp_uniform(0.1, 1),
        # 'colsample_bynode' : sp_uniform(0.1, 1)
        # 'data_sample_strategy' : ['bagging', 'goss'],
        # 'num_leaves': sp_randint(1, 200),
        # 'drop_rate': sp_uniform(0, 1),
        # 'skip_drop': sp_uniform(0, 1),
        # 'min_data_per_group': sp_randint(10, 200),
        # 'max_cat_threshold': sp_randint(10, 100),
        # 'cat_l2': sp_randint(10, 100),
        # 'cat_smooth': sp_randint(10, 100),
    }

    # Create a LightGBM regressor object
    lgb_reg = lgb.LGBMRegressor(**base_params)

    # Create a RandomizedSearchCV object
    random_search = HalvingRandomSearchCV(estimator=lgb_reg, param_distributions=param_dist,
                                       scoring='neg_mean_absolute_error',
                                       cv=cv, random_state=222, verbose=1,
                                         aggressive_elimination= True,
                                         max_resources=10000, min_resources=5)

    # Fit the random search to the data
    random_search.fit(X_train, y_train, categorical_feature=cat_features)

    # Return the best estimator and best parameters
    return random_search.best_estimator_, random_search.best_params_

base_params_p1 = {
    'verbose': -1,
    'metric': 'mae',
    'n_jobs': 32,
    'boosting': 'dart',
    'objective': 'tweedie'
}

i=4
train = processed_df_no_na[(date_filter <= to_datetime('2023-04-30'))]
val = processed_df_no_na[(date_filter > to_datetime('2023-04-30'))]
print(f"Fold {i}")
print(f"Train rows: {len(train)}")
print(f"Val rows: {len(val)}")

target_cols = ['target', 'target_installed_capacity']
drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
            'snowfall_fw', 'snowfall_hw_means']

df_train_target = train[target_cols]
df_train_data = train.drop(drop_cols, axis=1)

df_val_target2 = val[target_cols]
df_val_data2 = val.drop(drop_cols, axis=1)

cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
cat_features = [c for c in cat_features if c in df_train_data.columns]

# Fit the model
best_model, best_params = tune_lgbm_model(base_params_p1, df_train_data, df_train_target["target"])

print("Best parameters of p1:", best_params)

Fold 4
Train rows: 1901926
Val rows: 99168
n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 5
max_resources_: 10000
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 2000
n_resources: 5
Fitting 3 folds for each of 2000 candidates, totalling 6000 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
1248 fits failed out of a total of 6000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
627 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\sklearn.py", line 1049, in fit
    super().fit(
  File "C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\sklearn.py", line 842, in fit
    self._Booster = train(
  File "C:\Users\mskel\.conda\en

----------
iter: 1
n_candidates: 667
n_resources: 15
Fitting 3 folds for each of 667 candidates, totalling 2001 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [-464.41733389 -464.41733389           nan ... -245.03820658 -245.03820658
 -245.03820658]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [-385.54118533 -385.54118533           nan ... -395.35291303 -395.35291303
 -395.35291303]
  warnings.warn(


----------
iter: 2
n_candidates: 223
n_resources: 45
Fitting 3 folds for each of 223 candidates, totalling 669 fits


...

----------
iter: 5
n_candidates: 9
n_resources: 1215
Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [-464.41733389 -464.41733389           nan ... -302.70937357 -298.30003707
 -102.15361065]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [-385.54118533 -385.54118533           nan ... -260.13046849 -255.78020315
  -55.86822855]
  warnings.warn(


----------
iter: 6
n_candidates: 3
n_resources: 3645
Fitting 3 folds for each of 3 candidates, totalling 9 fits


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [-464.41733389 -464.41733389           nan ... -267.88819076 -276.470062
  -90.02948585]
  warnings.warn(
C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the train scores are non-finite: [-385.54118533 -385.54118533           nan ... -238.59997622 -247.26112996
  -53.62787524]
  warnings.warn(


Best parameters of p1: {'colsample_bynode': 0.3104645962074113, 'colsample_bytree': 0.8812805682576964, 'lambda_l1': 1.4774526938254677, 'lambda_l2': 0.9829140928475261, 'learning_rate': 0.020111960332727104, 'max_bin': 497, 'min_data_in_leaf': 41, 'n_estimators': 2279}
CPU times: total: 10h 7min 5s
Wall time: 20min 13s


In [32]:
best_model2 = best_model

In [38]:
importance = pd.DataFrame({'importance':best_model.feature_importances_, 'name':best_model.feature_name_})
importance = importance.sort_values('importance', ascending=False)
importance.tail(50)

,importance,name
307,2,highest_price_7d_avg_log
267,2,winddirection_10m_hw_means_hw_lagged_log
283,2,direct_solar_radiation_hw_variances_hw_lagged_log
220,2,cloudcover_total_hw_means_log
254,2,shortwave_radiation_hw_lagged_log
255,2,direct_solar_radiation_hw_lagged_log
226,2,shortwave_radiation_hw_means_log
262,2,cloudcover_total_hw_means_hw_lagged_log
238,2,windspeed_10m_hw_variances_log
248,2,cloudcover_total_hw_lagged_log


In [61]:
%%time

# https://www.kaggle.com/code/chaozhuang/enefit-eda-w-fft-ssa-arima-lgbm?scriptVersionId=156414824#Predictive-Modelling
from lightgbm import LGBMRegressor
import random
import lightgbm as lgb
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

def tune_lgbm_model(base_params, X_train, y_train, n_iter=8, cv=3):
    """
    Tune a LightGBM model based on a base set of parameters.

    :param base_params: Dictionary of base parameters for the model
    :param X_train: Training features
    :param y_train: Training target variable
    :param n_iter: Number of iterations for RandomizedSearchCV
    :param cv: Number of cross-validation folds
    :return: Best estimator and best parameters
    """
    # Parameter distributions for random search
    cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
    cat_features = [c for c in cat_features if c in df_train_data.columns]
    
    param_dist = {
        'learning_rate': sp_uniform(0.005, 0.5),
        'lambda_l1': sp_uniform(0, 5), 
        'lambda_l2': sp_uniform(0, 5), 
        'max_bin': sp_randint(100, 1000),
        'min_data_in_leaf': sp_randint(15, 300),
        'n_estimators': sp_randint(1000, 3500),
        
        # 'colsample_bytree' : sp_uniform(0.1, 1),
        # 'colsample_bynode' : sp_uniform(0.1, 1)
        # 'data_sample_strategy' : ['bagging', 'goss'],
        # 'num_leaves': sp_randint(1, 200),
        # 'drop_rate': sp_uniform(0, 1),
        # 'skip_drop': sp_uniform(0, 1),
        # 'min_data_per_group': sp_randint(10, 200),
        # 'max_cat_threshold': sp_randint(10, 100),
        # 'cat_l2': sp_randint(10, 100),
        # 'cat_smooth': sp_randint(10, 100),
    }

    # Create a LightGBM regressor object
    lgb_reg = lgb.LGBMRegressor(**base_params)

    # Create a RandomizedSearchCV object
    random_search = HalvingRandomSearchCV(estimator=lgb_reg, param_distributions=param_dist,
                                       scoring='neg_mean_absolute_error',
                                       cv=cv, random_state=222, verbose=1,
                                         aggressive_elimination= True,
                                         max_resources=15000, min_resources=5)

    # Fit the random search to the data
    
    random_search.fit(X_train, y_train, categorical_feature=cat_features)

    # Return the best estimator and best parameters
    return random_search.best_estimator_, random_search.best_params_

base_params_p1 = {
    'verbose': -1,
    'metric': 'mae',
    'n_jobs': 32,
    'boosting': 'dart',
    'objective': 'tweedie'
}

i=4
train = processed_df_no_na#[(date_filter <= to_datetime('2023-03-30'))]
val = processed_df_no_na[(date_filter > to_datetime('2023-03-30'))]
print(f"Fold {i}")
print(f"Train rows: {len(train)}")
print(f"Val rows: {len(val)}")

target_cols = ['target', 'target_installed_capacity']
drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
            'snowfall_fw', 'snowfall_hw_means']

with open('data/excluded.txt', 'r') as f:
    txt = f.read()
    txt_lst = txt.split('\n')
drop_cols += txt_lst


df_train_target = train[target_cols]
drop_cols = [c for c in drop_cols if c in train.columns]
print(drop_cols)
df_train_data = train.drop(drop_cols, axis=1)

df_val_target2 = val[target_cols]
drop_cols = [c for c in drop_cols if c in val.columns]
df_val_data2 = val.drop(drop_cols, axis=1)

cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
cat_features = [c for c in cat_features if c in df_train_data.columns]

# Fit the model
best_model, best_params = tune_lgbm_model(base_params_p1, df_train_data, df_train_target["target"])

print("Best parameters of p1:", best_params)

Fold 4
Train rows: 2001094
Val rows: 195264
['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances', 'snowfall_fw', 'snowfall_hw_means', 'eic_count_log', 'rain_hw_lagged', 'cloudcover_high_hw_lagged', 'target_lag_19h_log', 'rain_hw_lagged_log', 'target_lag_18h_log', 'target_lag_17h_log', 'snowfall_hw_means_hw_lagged', 'cloudcover_high_hw_means_hw_lagged', 'shortwave_radiation_hw_means_hw_lagged', 'target_lag_7h_log', 'target_lag_4h_log', 'windspeed_10m_hw_variances_hw_lagged', 'diffuse_radiation_hw_variances_hw_lagged', 'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_total_fw', '10_metre_u_wind_component', 'windspeed_10m_log', 'winddirection_10m_log', 'shortwave_radiation_log', 'direct_solar_radiation_log', 'dewpoint_hw_lagged_log', 'temperature_hw_lagged_log', 'diffuse_radiation_hw_variances_log', 'direct_solar_radiation_hw_v

In [42]:
with open('data/excluded.txt', 'r') as f:
    txt = f.read()
    txt_lst = txt.split('\n')
txt_lst

['eic_count_log',
 'rain_hw_lagged',
 'cloudcover_high_hw_lagged',
 'target_lag_19h_log',
 'rain_hw_lagged_log',
 'target_lag_18h_log',
 'target_lag_17h_log',
 'snowfall_hw_means_hw_lagged',
 'cloudcover_high_hw_means_hw_lagged',
 'shortwave_radiation_hw_means_hw_lagged',
 'target_lag_7h_log',
 'target_lag_4h_log',
 'windspeed_10m_hw_variances_hw_lagged',
 'diffuse_radiation_hw_variances_hw_lagged',
 'cloudcover_high_fw',
 'cloudcover_low_fw',
 'cloudcover_total_fw',
 '10_metre_u_wind_component',
 'windspeed_10m_log',
 'winddirection_10m_log',
 'shortwave_radiation_log',
 'direct_solar_radiation_log',
 'dewpoint_hw_lagged_log',
 'temperature_hw_lagged_log',
 'diffuse_radiation_hw_variances_log',
 'direct_solar_radiation_hw_variances_log',
 'shortwave_radiation_hw_variances_log',
 'winddirection_10m_hw_variances_log',
 'windspeed_10m_hw_variances_log',
 'cloudcover_high_hw_variances_log',
 'cloudcover_mid_hw_variances_log',
 'surface_pressure_hw_variances_log',
 'rain_hw_variances_log',

In [62]:
from lightgbm import LGBMRegressor

def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        target_cols = ['target', 'target_installed_capacity']
        drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                    'snowfall_fw', 'snowfall_hw_means']
        with open('data/excluded.txt', 'r') as f:
            txt = f.read()
            txt_lst = txt.split('\n')
        drop_cols += txt_lst
        
        
        df_train_target = train[target_cols]
        drop_cols = [c for c in drop_cols if c in train.columns]
        print(drop_cols)
        df_train_data = train.drop(drop_cols, axis=1)
        
        df_val_target2 = val[target_cols]
        drop_cols = [c for c in drop_cols if c in val.columns]
        df_val_data2 = val.drop(drop_cols, axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        cat_features = [c for c in cat_features if c in df_train_data.columns]
        
        # We leave max_depth as -1
        # Tune num_leaves, default is 31, let's double it
        params = {'lambda_l1': 2.567273484961577, 'lambda_l2': 0.8717364468687694, 'learning_rate': 0.3675157721473389, 'max_bin': 993, 'min_data_in_leaf': 114, 'n_estimators': 1535,
                  'objective': 'regression_l1', 'reg_sqrt': False}
        clf2 = LGBMRegressor(**params, random_state=42, verbose=0, n_jobs=22, importance_type='gain')
        clf2.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

        from sklearn.metrics import mean_absolute_error
        
        print("###############   Target   #################")
        y_pred = clf2.predict(df_train_data)
        y_pred
        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f"For fold {i}: Train Mean Absolute Error:", mae)

        y_pred_val = clf2.predict(df_val_data2)
        y_pred_val

        mae = mean_absolute_error(df_val_target2.target, y_pred_val)
        print(f"For fold {i}: Fold Val Mean Absolute Error:", mae)
        
        importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        display(importance.head(50))
        display(importance.tail(50))
        print()
        print()

In [63]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264
['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances', 'snowfall_fw', 'snowfall_hw_means', 'eic_count_log', 'rain_hw_lagged', 'cloudcover_high_hw_lagged', 'target_lag_19h_log', 'rain_hw_lagged_log', 'target_lag_18h_log', 'target_lag_17h_log', 'snowfall_hw_means_hw_lagged', 'cloudcover_high_hw_means_hw_lagged', 'shortwave_radiation_hw_means_hw_lagged', 'target_lag_7h_log', 'target_lag_4h_log', 'windspeed_10m_hw_variances_hw_lagged', 'diffuse_radiation_hw_variances_hw_lagged', 'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_total_fw', '10_metre_u_wind_component', 'windspeed_10m_log', 'winddirection_10m_log', 'shortwave_radiation_log', 'direct_solar_radiation_log', 'dewpoint_hw_lagged_log', 'temperature_hw_lagged_log', 'diffuse_radiation_hw_variances_log', 'direct_solar_radiation_hw_v

,importance,name
38,1.972470e+08,target_rolling_avg_hour_7d
112,8.551790e+05,surface_solar_radiation_downwards
46,4.271994e+05,eic_count
0,4.212776e+05,county
137,3.428619e+05,day_of_year_sin
53,3.001724e+05,cloudcover_total
151,2.833739e+05,target_lag_72h_log
27,2.533841e+05,target_lag_23h
59,2.327026e+05,shortwave_radiation
30,2.208945e+05,target_lag_72h


,importance,name
79,622.518606,diffuse_radiation_hw_variances
57,621.342197,windspeed_10m
20,620.446590,target_lag_16h
99,617.291691,temperature_hw_variances_hw_lagged
11,611.159897,target_lag_7h
135,564.851990,hour_sin
142,563.830894,target_lag_10h_log
103,546.467400,cloudcover_mid_hw_variances_hw_lagged
75,511.625389,cloudcover_total_hw_variances
58,491.260601,winddirection_10m




Fold 1
Train rows: 1304266
Val rows: 173328
['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances', 'snowfall_fw', 'snowfall_hw_means', 'eic_count_log', 'rain_hw_lagged', 'cloudcover_high_hw_lagged', 'target_lag_19h_log', 'rain_hw_lagged_log', 'target_lag_18h_log', 'target_lag_17h_log', 'snowfall_hw_means_hw_lagged', 'cloudcover_high_hw_means_hw_lagged', 'shortwave_radiation_hw_means_hw_lagged', 'target_lag_7h_log', 'target_lag_4h_log', 'windspeed_10m_hw_variances_hw_lagged', 'diffuse_radiation_hw_variances_hw_lagged', 'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_total_fw', '10_metre_u_wind_component', 'windspeed_10m_log', 'winddirection_10m_log', 'shortwave_radiation_log', 'direct_solar_radiation_log', 'dewpoint_hw_lagged_log', 'temperature_hw_lagged_log', 'diffuse_radiation_hw_variances_log', 'direct_solar_radiation_hw

,importance,name
38,2.110247e+08,target_rolling_avg_hour_7d
112,9.635458e+06,surface_solar_radiation_downwards
4,1.044015e+06,target_rt
170,7.662132e+05,surface_solar_radiation_downwards_log
54,6.090403e+05,cloudcover_low
0,4.857442e+05,county
30,4.516469e+05,target_lag_72h
130,4.145994e+05,hour
59,3.735688e+05,shortwave_radiation
46,3.308445e+05,eic_count


...

,importance,name
38,2.815557e+08,target_rolling_avg_hour_7d
112,1.065112e+07,surface_solar_radiation_downwards
4,1.346191e+06,target_rt
137,6.321508e+05,day_of_year_sin
5,5.663187e+05,target_lag_1h
46,5.032342e+05,eic_count
0,4.377484e+05,county
30,4.164154e+05,target_lag_72h
151,3.669761e+05,target_lag_72h_log
59,2.648487e+05,shortwave_radiation


,importance,name
102,804.474403,surface_pressure_hw_variances_hw_lagged
169,790.602982,10_metre_u_wind_component_log
117,764.533310,yesterdays_euros_per_mwh
162,751.167397,dewpoint_hw_means_hw_lagged_log
75,734.684288,cloudcover_total_hw_variances
153,723.428711,target_lag_120h_log
106,709.424500,shortwave_radiation_hw_variances_hw_lagged
144,705.728107,target_lag_14h_log
109,676.365211,dewpoint_fw
71,657.096210,diffuse_radiation_hw_means




Fold 4
Train rows: 1824598
Val rows: 176496
['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances', 'snowfall_fw', 'snowfall_hw_means', 'eic_count_log', 'rain_hw_lagged', 'cloudcover_high_hw_lagged', 'target_lag_19h_log', 'rain_hw_lagged_log', 'target_lag_18h_log', 'target_lag_17h_log', 'snowfall_hw_means_hw_lagged', 'cloudcover_high_hw_means_hw_lagged', 'shortwave_radiation_hw_means_hw_lagged', 'target_lag_7h_log', 'target_lag_4h_log', 'windspeed_10m_hw_variances_hw_lagged', 'diffuse_radiation_hw_variances_hw_lagged', 'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_total_fw', '10_metre_u_wind_component', 'windspeed_10m_log', 'winddirection_10m_log', 'shortwave_radiation_log', 'direct_solar_radiation_log', 'dewpoint_hw_lagged_log', 'temperature_hw_lagged_log', 'diffuse_radiation_hw_variances_log', 'direct_solar_radiation_hw

,importance,name
38,3.147001e+08,target_rolling_avg_hour_7d
112,2.157624e+06,surface_solar_radiation_downwards
30,8.015237e+05,target_lag_72h
130,7.873925e+05,hour
137,7.426940e+05,day_of_year_sin
46,5.753586e+05,eic_count
60,4.189572e+05,direct_solar_radiation
5,3.833462e+05,target_lag_1h
35,2.518593e+05,target_lag_264h
0,2.422683e+05,county


,importance,name
147,874.895996,target_lag_21h_log
113,866.893600,total_precipitation
9,861.403992,target_lag_5h
75,843.988007,cloudcover_total_hw_variances
66,838.258003,cloudcover_total_hw_means
21,796.508797,target_lag_17h
43,788.959000,target_rolling_avg_hour_7d_estonia
8,770.992004,target_lag_4h
117,744.381500,yesterdays_euros_per_mwh
78,740.043991,direct_solar_radiation_hw_variances


In [22]:
from lightgbm import LGBMRegressor

def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        target_cols = ['target', 'target_installed_capacity']
        drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                    'snowfall_fw', 'snowfall_hw_means']
        
        df_train_target = train[target_cols]
        df_train_data = train.drop(drop_cols, axis=1)
        
        df_val_target2 = val[target_cols]
        df_val_data2 = val.drop(drop_cols, axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        cat_features = [c for c in cat_features if c in df_train_data.columns]
        
        # We leave max_depth as -1
        # Tune num_leaves, default is 31, let's double it
        params = {'boosting': 'dart', 'lambda_l1': 0.5495562797837548, 'lambda_l2': 1.162175586039744, 'learning_rate': 0.27395000326947383, 'max_bin': 466, 'min_data_in_leaf': 136, 'n_estimators': 6132, 'num_iterations': 1891, 'objective': 'tweedie', 'reg_sqrt': True}
        clf2 = LGBMRegressor(**params, random_state=42, verbose=0, n_jobs=15, importance_type='gain')
        clf2.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

        from sklearn.metrics import mean_absolute_error
        
        print("###############   Target   #################")
        y_pred = clf2.predict(df_train_data)
        y_pred
        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f"For fold {i}: Train Mean Absolute Error:", mae)

        y_pred_val = clf2.predict(df_val_data2)
        y_pred_val

        mae = mean_absolute_error(df_val_target2.target, y_pred_val)
        print(f"For fold {i}: Fold Val Mean Absolute Error:", mae)
        
        importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        # display(importance.head(30))
        # display(importance.tail(30))
        print()
        print()

In [23]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

...

[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

In [20]:
from lightgbm import LGBMRegressor

def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        target_cols = ['target', 'target_installed_capacity']
        drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                    'snowfall_fw', 'snowfall_hw_means']
        
        df_train_target = train[target_cols]
        df_train_data = train.drop(drop_cols, axis=1)
        
        df_val_target2 = val[target_cols]
        df_val_data2 = val.drop(drop_cols, axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        cat_features = [c for c in cat_features if c in df_train_data.columns]
        
        # We leave max_depth as -1
        # Tune num_leaves, default is 31, let's double it
        params = {'boosting': 'dart', 'lambda_l1': 0.9463464945326672, 'lambda_l2': 0.5311225677898304, 'learning_rate': 0.42981688122448874, 'max_bin': 701, 'min_data_in_leaf': 295, 'n_estimators': 6289, 
                  'num_iterations': 1887, 'objective': 'tweedie', 'reg_sqrt': True}
        clf2 = LGBMRegressor(**params, random_state=42, verbose=0, n_jobs=32, importance_type='gain')
        clf2.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

        from sklearn.metrics import mean_absolute_error
        
        print("###############   Target   #################")
        y_pred = clf2.predict(df_train_data)
        y_pred
        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f"For fold {i}: Train Mean Absolute Error:", mae)

        y_pred_val = clf2.predict(df_val_data2)
        y_pred_val

        mae = mean_absolute_error(df_val_target2.target, y_pred_val)
        print(f"For fold {i}: Fold Val Mean Absolute Error:", mae)
        
        importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        # display(importance.head(30))
        # display(importance.tail(30))
        print()
        print()

In [21]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[L

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[L

...

[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[L

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[L

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[LightGBM] [Warning] min_data_in_leaf is set=295, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=295
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.9463464945326672, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9463464945326672
[LightGBM] [Warning] lambda_l2 is set=0.5311225677898304, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5311225677898304
[L

In [22]:
from lightgbm import LGBMRegressor

def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        target_cols = ['target', 'target_installed_capacity']
        drop_cols = ['target', 'target_installed_capacity', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                    'snowfall_fw', 'snowfall_hw_means']
        
        df_train_target = train[target_cols]
        df_train_data = train.drop(drop_cols, axis=1)
        
        df_val_target2 = val[target_cols]
        df_val_data2 = val.drop(drop_cols, axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        cat_features = [c for c in cat_features if c in df_train_data.columns]
        
        # We leave max_depth as -1
        # Tune num_leaves, default is 31, let's double it
        params = {'boosting': 'dart', 'lambda_l1': 0.5350237641949025, 'lambda_l2': 0.2952089055653504, 'learning_rate': 0.11903395875401224, 'max_bin': 638, 'min_data_in_leaf': 145, 'n_estimators': 5120, 
                  'num_iterations': 2053, 'objective': 'tweedie', 'reg_sqrt': True}
        clf2 = LGBMRegressor(**params, random_state=42, verbose=0, n_jobs=32, importance_type='gain')
        clf2.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

        from sklearn.metrics import mean_absolute_error
        
        print("###############   Target   #################")
        y_pred = clf2.predict(df_train_data)
        y_pred
        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f"For fold {i}: Train Mean Absolute Error:", mae)

        y_pred_val = clf2.predict(df_val_data2)
        y_pred_val

        mae = mean_absolute_error(df_val_target2.target, y_pred_val)
        print(f"For fold {i}: Fold Val Mean Absolute Error:", mae)
        
        importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        # display(importance.head(30))
        # display(importance.tail(30))
        print()
        print()

In [23]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264


C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

...

[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.5495562797837548, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5495562797837548
[LightGBM] [Warning] lambda_l2 is set=1.162175586039744, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.162175586039744
[Light

Best Parameter runs:


```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 1.7896547008552977, 'lambda_l2': 1.1957999576221703, 'learning_rate': 0.1882811352534675, 'max_bin': 314, 'min_data_in_leaf': 54, 'num_iterations': 824} = LGBMRegressor(random_state=42, boosting='dart', n_estimators=2500, verbose=0, n_jobs=15, objective='l2', importance_type='gain', lambda_l1= 1.7896547008552977, lambda_l2= 1.1957999576221703, learning_rate= 0.1882811352534675, max_bin= 314, min_data_in_leaf= 54, num_iterations= 824)```

```Best parameters of p1: {'boosting': 'gbrt', 'lambda_l1': 0.34672930701554416, 'lambda_l2': 0.7821212151464816, 'learning_rate': 0.07733541316820935, 'max_bin': 487, 'min_data_in_leaf': 16, 'n_estimators': 2021, 'num_iterations': 489, 'objective': 'tweedie', 'reg_sqrt': False}```

```Best parameters of p1: {'boosting': 'gbrt', 'lambda_l1': 1.9207022987271727, 'lambda_l2': 0.10588062856381697, 'learning_rate': 0.12891341726246747, 'max_bin': 401, 'min_data_in_leaf': 76, 'n_estimators': 2300, 'num_iterations': 730, 'objective': 'tweedie', 'reg_sqrt': True}```

```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 1.1928653738419932, 'lambda_l2': 1.6746474446216606, 'learning_rate': 0.2727459158987165, 'max_bin': 145, 'min_data_in_leaf': 122, 'n_estimators': 6128, 'num_iterations': 1765, 'objective': 'tweedie', 'reg_sqrt': False}```

```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 0.6864834137130547, 'lambda_l2': 0.505513780743504, 'learning_rate': 0.2549750586501872, 'max_bin': 467, 'min_data_in_leaf': 107, 'n_estimators': 3559, 'num_iterations': 1142, 'objective': 'regression_l1', 'reg_sqrt': False}```

```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 0.5495562797837548, 'lambda_l2': 1.162175586039744, 'learning_rate': 0.27395000326947383, 'max_bin': 466, 'min_data_in_leaf': 136, 'n_estimators': 6132, 'num_iterations': 1891, 'objective': 'tweedie', 'reg_sqrt': True}```

```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 0.9463464945326672, 'lambda_l2': 0.5311225677898304, 'learning_rate': 0.42981688122448874, 'max_bin': 701, 'min_data_in_leaf': 295, 'n_estimators': 6289, 'num_iterations': 1887, 'objective': 'tweedie', 'reg_sqrt': True}```

```Best parameters of p1: {'boosting': 'dart', 'lambda_l1': 0.5350237641949025, 'lambda_l2': 0.2952089055653504, 'learning_rate': 0.11903395875401224, 'max_bin': 638, 'min_data_in_leaf': 145, 'n_estimators': 5120, 'num_iterations': 2053, 'objective': 'tweedie', 'reg_sqrt': True}```


```Best parameters of p1: {'lambda_l1': 1.1210308999481333, 'lambda_l2': 1.3202715549769235, 'learning_rate': 0.49368057640644114, 'max_bin': 285, 'min_data_in_leaf': 278, 'n_estimators': 4430, 'reg_sqrt': True, 'verbose': -1, 'metric': 'mae', 'n_jobs': 32, 'boosting': 'dart', 'objective': 'tweedie'}```

```Best parameters of p1: {'lambda_l1': 0.7466999841658806, 'lambda_l2': 3.2140838539606458, 'learning_rate': 0.13753679743025782, 'max_bin': 723, 'min_data_in_leaf': 150, 'n_estimators': 5593, 'verbose': -1, 'metric': 'mae', 'n_jobs': 32, 'boosting': 'dart', 'objective': 'tweedie'}```

```Best parameters of p1: {'lambda_l1': 2.931420537739911, 'lambda_l2': 4.510580686709511, 'learning_rate': 0.058926445091536876, 'max_bin': 961, 'min_data_in_leaf': 115, 'n_estimators': 5188} CPU times: total: 7h 43min 17s Wall time: 15min 5s```



Todo




```Best parameters of p1: {'colsample_bynode': 0.870026791381559, 'colsample_bytree': 0.21026509011420783, 'lambda_l1': 2.383100182236077, 'lambda_l2': 4.450575103732394, 'learning_rate': 0.2509071472970593, 'max_bin': 556, 'min_data_in_leaf': 133, 'n_estimators': 4594}
CPU times: total: 4h 30min 19s
Wall time: 8min 48s```

```Best parameters of p1: {'colsample_bynode': 0.5253603437351759, 'colsample_bytree': 0.6267686233402383, 'lambda_l1': 0.5452526132258517, 'lambda_l2': 1.1998362529626179, 'learning_rate': 0.32940999216169664, 'max_bin': 635, 'min_data_in_leaf': 100, 'n_estimators': 5522}
CPU times: total: 12h 4min 27s
Wall time: 24min 15s```

### Models

In [53]:
params={'n_iter': 2500,'verbose': 1,'objective': 'l2','metric': 'mae','learning_rate': 0.05073909898961407, 'colsample_bytree': 0.726023996436955, 'colsample_bynode': 0.5803681307354022, 
        'lambda_l1': 8.562963348932286, 'lambda_l2': 4.893256185259296, 'min_data_in_leaf': 115, 'max_depth': 23, 'num_leaves':50, 'max_bin': 898}

train = processed_df_no_na[date_filter <= datetime_cv_ranges[-1][0]]
val = processed_df_no_na[(date_filter <= datetime_cv_ranges[-1][1]) & (date_filter > datetime_cv_ranges[-1][0])]

df_train_target = train[['target', 'target_installed_capacity']]
df_train_data = train.drop(['target', 'target_installed_capacity'], axis=1)

df_val_target = val[['target', 'target_installed_capacity']]
df_val_data = val.drop(['target', 'target_installed_capacity'], axis=1)
        
clf = LGBMRegressor(**params, random_state=42)


cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])

clf.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

y_pred = clf.predict(df_train_data)
y_pred

from sklearn.metrics import mean_absolute_error

# Assuming you have two pandas Series: y_true and y_pred
y_pred = clf.predict(df_train_data)
mae = mean_absolute_error(df_train_target.target, y_pred)
print(f" Train Mean Absolute Error:", mae)

y_pred_val = clf.predict(df_val_data)
y_pred_val

mae = mean_absolute_error(df_val_target.target, y_pred_val)
print("Val Mean Absolute Error:", mae)

# y_pred_test = clf.predict(df_test_data)
# y_pred_test

importance = pd.DataFrame({'importance':clf.feature_importances_, 'name':clf.feature_name_})
importance = importance.sort_values('importance', ascending=False)
display(importance.head(10))
display(importance.tail(10))
print()
print()

C:\Users\mskel\.conda\envs\kaggle310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=2500, n_iter=2500 will be ignored. Current value: num_iterations=2500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.219732 seconds.
You can s

,importance,name
6,5715,target_rolling_avg_hour_7d
5,4274,target_rolling_avg_24h
126,4055,hour
12,4026,installed_capacity
7,3947,target_rolling_avg_hour_hour_day_4w
11,3284,eic_count
4,3144,target_rt
0,2969,county
8,2359,target_rolling_allp_avg_24h
9,2265,target_rolling_allp_avg_hour_7d


,importance,name
123,55,year
138,32,season
127,24,quarter
132,21,is_month_start
58,18,snowfall_hw_lagged
133,5,is_month_end
135,3,is_quarter_end
134,3,is_quarter_start
136,0,is_year_start
137,0,is_year_end


In [54]:
test_submission = pd.read_csv("data/example_test_files/sample_submission.csv")
test_submission

,row_id,data_block_id,target
0,2005872,634,0
1,2005873,634,0
2,2005874,634,0
3,2005875,634,0
4,2005876,634,0
...,...,...,...
12475,2018347,637,0
12476,2018348,637,0
12477,2018349,637,0
12478,2018350,637,0


In [55]:
clf.predict(df_val_data)

[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296


array([6.39107379e+00, 1.01617424e+03, 6.95058781e-02, ...,
       4.89672200e+01, 5.24024018e+00, 2.66437232e+02])

In [61]:
from data import public_timeseries_testing_util as enefit

with open('data_processor_testing.pkl', 'rb') as f:
    data_processor = pickle.load(f)
data_processor.df

env = enefit.make_env()

for (test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices, sample_submission) in env.iter_test():
    test_data = data_processor.process_test_data_timestep(test, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices)
    display(test_data)
    
    test_data_filtered = test_data[~test_data.currently_scored.fillna(True)]
    test_data_filtered = test_data_filtered.drop('target', axis=1)
    other_cols = test_data_filtered[['prediction_datetime', 'currently_scored', 'row_id']]
    test_data_filtered = test_data_filtered.drop(['prediction_datetime', 'currently_scored', 'row_id'], axis=1)
    preds = clf.predict(test_data_filtered)
    submission = other_cols[['row_id']].copy()
    submission['target'] = preds
    submission = submission.reset_index(drop=True)
    env.predict(submission)
    data_processor.test_orig_dfs[0]['currently_scored'] = True
    display(submission)

C:\Users\mskel\AppData\Local\Temp\ipykernel_6920\1522147731.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = df['datetime'] + dt.timedelta(days=1)


0


,county,is_business,product_type,target,is_consumption,row_id,prediction_datetime,currently_scored,target_rt,target_rolling_avg_24h,...,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos,is_ee_holiday
0,0,0,1,2.977,0,1960760,NaN,NaN,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
1,0,0,1,601.482,1,1960761,NaN,NaN,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
2,0,0,2,0.000,0,1960762,NaN,NaN,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
3,0,0,2,9.943,1,1960763,NaN,NaN,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
4,0,0,3,50.278,0,1960764,NaN,NaN,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51347,15,1,1,NaN,1,2008989,2023-05-28 23:00:00,False,32.809,26.492250,...,False,False,False,False,2,-0.258819,0.965926,0.575190,-0.818020,True
51348,15,1,3,NaN,0,2008990,2023-05-28 23:00:00,False,0.000,362.492542,...,False,False,False,False,2,-0.258819,0.965926,0.575190,-0.818020,True
51349,15,1,3,NaN,0,2008990,2023-05-28 23:00:00,False,0.000,375.690208,...,False,False,False,False,2,-0.258819,0.965926,0.575190,-0.818020,True
51350,15,1,3,NaN,1,2008991,2023-05-28 23:00:00,False,195.707,299.014875,...,False,False,False,False,2,-0.258819,0.965926,0.575190,-0.818020,True


[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296


,row_id,target
0,2005872,18.606309
1,2005872,26.932363
2,2005873,574.233623
3,2005873,516.410356
4,2005874,-2.070909
...,...,...
6235,2008989,37.287530
6236,2008990,-8.324577
6237,2008990,-8.324577
6238,2008991,288.868177


...

C:\Users\mskel\AppData\Local\Temp\ipykernel_6920\1522147731.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = df['datetime'] + dt.timedelta(days=1)


0


,county,is_business,product_type,target,is_consumption,row_id,prediction_datetime,currently_scored,target_rt,target_rolling_avg_24h,...,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos,is_ee_holiday
0,0,0,1,2.977,0,1960760,NaN,True,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
1,0,0,1,601.482,1,1960761,NaN,True,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
2,0,0,2,0.000,0,1960762,NaN,True,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
3,0,0,2,9.943,1,1960763,NaN,True,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
4,0,0,3,50.278,0,1960764,NaN,True,NaN,NaN,...,False,False,False,False,2,-0.258819,0.965926,0.764891,-0.644159,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70067,15,1,1,NaN,1,2018349,2023-05-31 23:00:00,False,31.484,33.753875,...,False,False,False,False,2,-0.258819,0.965926,0.532227,-0.846602,False
70068,15,1,3,NaN,0,2018350,2023-05-31 23:00:00,False,0.000,403.044625,...,False,False,False,False,2,-0.258819,0.965926,0.532227,-0.846602,False
70069,15,1,3,NaN,0,2018350,2023-05-31 23:00:00,False,0.000,286.517708,...,False,False,False,False,2,-0.258819,0.965926,0.532227,-0.846602,False
70070,15,1,3,NaN,1,2018351,2023-05-31 23:00:00,False,183.756,105.720042,...,False,False,False,False,2,-0.258819,0.965926,0.532227,-0.846602,False


[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296


,row_id,target
0,2015232,1.443170
1,2015232,1.150358
2,2015233,526.388732
3,2015233,545.576868
4,2015234,-8.904476
...,...,...
6235,2018349,40.218498
6236,2018350,2.111896
6237,2018350,-4.018734
6238,2018351,266.094187


In [62]:
def inverse_tic(preds, train):
    return preds/1000 * train.installed_capacity

def train_cv(df):
    for i in range(5):
        train = df[date_filter <= datetime_cv_ranges[i][0]]
        val = df[(date_filter <= datetime_cv_ranges[i][1]) & (date_filter > datetime_cv_ranges[i][0])]
        print(f"Fold {i}")
        print(f"Train rows: {len(train)}")
        print(f"Val rows: {len(val)}")
        
        df_train_target = train[['target', 'target_installed_capacity']]
        df_train_data = train.drop(['target', 'target_installed_capacity'], axis=1)
        
        df_val_target = val[['target', 'target_installed_capacity']]
        df_val_data = val.drop(['target', 'target_installed_capacity'], axis=1)
        
        cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
               'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
                'is_year_start', 'is_year_end', 'season', ] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
        
        clf = LGBMRegressor(random_state=42, n_estimators=1500, verbose=1, n_jobs=32, objective='l2', )
        clf_producer = LGBMRegressor(random_state=42, n_estimators=1500, verbose=1, n_jobs=32, objective='l2', )
        
        clf.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)
        clf_producer.fit(df_train_data[df_train_data.is_consumption==0], df_train_target[df_train_data.is_consumption==0].target, categorical_feature=cat_features)

        y_pred = clf.predict(df_train_data)
        y_pred_producer = clf_producer.predict(df_train_data[df_train_data.is_consumption==0])
        y_pred2 = y_pred.copy()
        y_pred2[df_train_data.is_consumption==0] = y_pred_producer 

        from sklearn.metrics import mean_absolute_error

        # Assuming you have two pandas Series: y_true and y_pred
        mae = mean_absolute_error(df_train_target.target, y_pred)
        print(f" Train Mean Absolute Error:", mae)
        mae = mean_absolute_error(df_train_target.target, y_pred2)
        print(f" Train Mean w Producer Absolute Error:", mae)

        y_pred_val = clf.predict(df_val_data)
        y_pred_val_producer = clf_producer.predict(df_val_data[df_val_data.is_consumption==0])
        y_pred_val2 = y_pred_val.copy()
        y_pred_val2[df_val_data.is_consumption==0] = y_pred_val_producer 

        mae = mean_absolute_error(df_val_target.target, y_pred_val)
        print("Val Mean Absolute Error:", mae)
        mae = mean_absolute_error(df_val_target.target, y_pred_val2)
        print("Val Mean w Producer Absolute Error:", mae)

        # y_pred_test = clf.predict(df_test_data)
        # y_pred_test

        # mae = mean_absolute_error(df_test_target.target, y_pred_test)
        # print("Test Mean Absolute Error:", mae)

        importance = pd.DataFrame({'importance':clf.feature_importances_, 'name':clf.feature_name_})
        importance = importance.sort_values('importance', ascending=False)
        # display(importance.head(10))
        # display(importance.tail(10))
        print()
        print()
        print()
        print()

In [63]:
train_cv(processed_df_no_na)

Fold 0
Train rows: 1129738
Val rows: 171264
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28482
[LightGBM] [Info] Number of data points in the train set: 1129738, number of used features: 167
[LightGBM] [Info] Start training from score 250.526332
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28481
[LightGBM] [Info] Number of data points in the train set: 564869, number of used features: 166
[LightGBM] [Info] Start training from score 87.581421
 Train Mean Absolute Error: 21.623644374411725
 Train Mean w Producer Absolute Error: 18.244623606390444
Val Mean Absolute Error: 45.406346380878844
Val Mean w Producer Absolute Error: 44.0577406652